In [2]:
#Import necessary modules
import pandas as pd
import numpy as np

#Load data
pymoli_df = pd.read_csv('Resources/purchase_data.csv')

## Player Count
>- Total number of players

In [3]:
#Self explanatory, total count of unique player names
total_count = pd.DataFrame([pymoli_df.SN.nunique()], columns=['Total Players'])
total_count

,Total Players
0,576


## Purchasing Analysis (Total)

In [4]:
#Gather series of unique item names
unique_items = pymoli_df['Item Name'].nunique()

#calc the average purchase price of all items into a series
avg_purch_price = "${:.2f}".format(pymoli_df['Price'].mean())

#calc the total number of purchases into a series
purchase_total_number = pymoli_df['Purchase ID'].count()

#calc the total revenue into a series
total_revenue = "${:.2f}".format(pymoli_df['Price'].sum())

#Make dataframe and show it
purchasing_analysis = pd.DataFrame({'Number of Unique Items': [unique_items],
                  'Average Purchase Price': [avg_purch_price],
                  'Total Number of Purchases': [purchase_total_number],
                  'Total Revenue': [total_revenue]})
purchasing_analysis

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


## Gender Demographics
>- Percentage and Count of male, female and other/non-disclosed players

In [5]:
#gather a series of unique players count
gender_countof_players = pymoli_df.groupby('Gender').SN.nunique()

#gather a series of the percentage calc
gender_percentof_players = round(pymoli_df.groupby('Gender').SN.nunique() / pymoli_df.SN.nunique() * 100, 2)

#make new frame and show it
gender_demographics_df = pd.DataFrame({
    'Total Count': gender_countof_players,
    'Percentage of Players': gender_percentof_players
})
gender_demographics_df

,Total Count,Percentage of Players
Gender,,
Female,81,14.06
Male,484,84.03
Other / Non-Disclosed,11,1.91


## Purchasing Analysis (Gender)
>- summary of purchase count, average purchase price, total purchase value and average purchase total per person by gender

In [6]:
#create frame with only the columns we need and grab the necessary series
gender_data_purchases = pymoli_df[['Gender', 'Purchase ID']]
gender_data_purchase_counts = gender_data_purchases.groupby('Gender').count()

#create another frame with only the columns we need and grab the necessary series
gender_data_purchase_avgs = pymoli_df[['Gender', 'Price']]
gender_data_price_avgs = gender_data_purchase_avgs.groupby('Gender').mean()

gender_data_price_total = gender_data_purchase_avgs.groupby('Gender').sum()

#gender_data_avg_per_person

In [7]:
# Make our final Df and show it
purchasing_analysis_gender = pd.DataFrame({
    'Purchase Count': gender_data_purchase_counts['Purchase ID'],
    'Average Purchase Price': gender_data_price_avgs['Price'].map("${:.2f}".format),
    'Total Purchase Value': gender_data_price_total['Price'].map("${:.2f}".format)
#    'Avg Total Purchase per Person': gd_price_total['Price'].map("${:.2f}".format) / 
})
purchasing_analysis_gender

,Purchase Count,Average Purchase Price,Total Purchase Value
Gender,,,
Female,113,$3.20,$361.94
Male,652,$3.02,$1967.64
Other / Non-Disclosed,15,$3.35,$50.19


##  Age Demographics
>- summary of players by age group

In [8]:
#First create bins and labels
bins = [0, 9, 14, 19, 24, 29, 34, 39, 200]

group_labels = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

In [9]:
#Next add our age group column
pymoli_df['age_group'] =  pd.cut(pymoli_df['Age'], bins, labels=group_labels)

In [10]:
#create our total counts of each bin series
unique_players_by_age = pymoli_df.groupby('age_group').SN.nunique()

#create dataframe to show total count and percentage of total players for each group
age_demographics = pd.DataFrame({
    'Total Count': unique_players_by_age,
    'Percentage of Players': (unique_players_by_age / pymoli_df.SN.nunique() * 100).map("{:.2f}".format)
})
age_demographics

,Total Count,Percentage of Players
age_group,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


##  Purchasing Analysis (Age)
>- summary of purchase count, average purchase price, total purchase value and average purchase total per person by age group

In [11]:
#grab the columns needed for this section in two frames
purchases_count_by_age = pymoli_df[['age_group', 'Purchase ID']]
purchases_values_by_age = pymoli_df[['age_group', 'Price']]

#totals of purchases by age
total_purchases_by_age = purchases_count_by_age.groupby('age_group').count() 

#Dollar values of purchases by age
total_purchases_values_by_age = purchases_values_by_age.groupby('age_group').sum()

#Show datafraame
purchasing_analysis_age = pd.DataFrame({
    'Purchase Count': total_purchases_by_age['Purchase ID'],
    'Average Purchase Price': (total_purchases_values_by_age['Price'] / total_purchases_by_age['Purchase ID']).map("${:.2f}".format),
    'Total Purchase Value': total_purchases_values_by_age['Price'].map("${:.2f}".format)
})
purchasing_analysis_age

,Purchase Count,Average Purchase Price,Total Purchase Value
age_group,,,
<10,23,$3.35,$77.13
10-14,28,$2.96,$82.78
15-19,136,$3.04,$412.89
20-24,365,$3.05,$1114.06
25-29,101,$2.90,$293.00
30-34,73,$2.93,$214.00
35-39,41,$3.60,$147.67
40+,13,$2.94,$38.24


##  Top Spenders
>- Stats on the top 5 spenders in the game

In [12]:
#grab the columns I need for this section
top_five = pymoli_df[['SN', 'Purchase ID', 'Price']]

#make frames of the groupings needed
top_five_sn_group = top_five.groupby('SN').count() #DF
top_five_pricetotal_group = top_five.groupby('SN')['Price'].sum().reset_index() #DF
top_five_pricetotal_group.set_index('SN', inplace=True)

#Merge the two frames to make the next frame needed
top_five_merged = top_five_sn_group.merge(top_five_pricetotal_group, how='outer', on='SN')
top_five_merged_rev = top_five_merged.drop(columns= 'Price_x')

top_five_merged_rev.rename(columns={'Purchase ID': 'Purchase Count',
                                   'Price_y': 'Total Purchase Value'}, inplace=True)
top_five_merged_rev['Average Purchase Price'] = \
top_five_merged_rev['Total Purchase Value'] / top_five_merged_rev['Purchase Count']

#sort the final frame and show only the top five

top_five_finalsorted = top_five_merged_rev.sort_values('Total Purchase Value', ascending=False)
top_five_finalsorted['Total Purchase Value'] = top_five_finalsorted['Total Purchase Value'].map("${:.2f}".format)
top_five_finalsorted['Average Purchase Price'] = top_five_finalsorted['Average Purchase Price'].map("${:.2f}".format)

top_five_finalsorted[:5]


,Purchase Count,Total Purchase Value,Average Purchase Price
SN,,,
Lisosia93,5,$18.96,$3.79
Idastidru52,4,$15.45,$3.86
Chamjask73,3,$13.83,$4.61
Iral74,4,$13.62,$3.40
Iskadarya95,3,$13.10,$4.37


##  Most Popular Items
>- Identify the 5 most popular items by purchase count

In [13]:
#Gather the columns needed for this section from the main DF
most_popular_items = pymoli_df[['Item ID', 'Item Name', 'Price']]

#Create a DF for total purchase value
most_popular_items_total_purch_val = most_popular_items.groupby(['Item ID', 'Item Name'])['Price'].sum().reset_index()
most_popular_items_total_purch_val.set_index('Item ID', inplace=True)
most_popular_items_total_purch_val.rename(columns={'Price' : 'Total Purchase Value'}, inplace=True)

##Create a DF for purchase count
most_popular_items_purchase_count = most_popular_items.groupby(['Item ID', 'Item Name'])['Price'].count().reset_index()
most_popular_items_purchase_count.set_index('Item ID', inplace=True)
most_popular_items_purchase_count.rename(columns={'Price' : 'Purchase Count'}, inplace=True)

#Merge the frames and clean up, add the item price back and show
most_popular_items_merge = most_popular_items_total_purch_val.merge(most_popular_items_purchase_count, on='Item ID')
most_popular_items_merge.drop(columns='Item Name_y', inplace=True)

#adding price back
most_popular_items_price = most_popular_items.groupby(['Item ID', 'Price'])['Item Name'].count().reset_index()
most_popular_items_merge['Price'] = most_popular_items_price['Price']

#Sort by purchase count
most_popular_items_sorted_by_purchase_count = most_popular_items_merge.sort_values('Purchase Count', ascending=False)
most_popular_items_sorted_by_purchase_count['Total Purchase Value'] = \
most_popular_items_sorted_by_purchase_count['Total Purchase Value'].map("${:.2f}".format)
most_popular_items_sorted_by_purchase_count['Price'] = \
most_popular_items_sorted_by_purchase_count['Price'].map("${:.2f}".format)
most_popular_items_sorted_by_purchase_count.rename(columns={'Item Name_x': 'Item Name'}, inplace=True) #Last clean up

most_popular_items_sorted_by_purchase_count[:5]

,Item Name,Total Purchase Value,Purchase Count,Price
Item ID,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",$50.76,12,$4.48
145,Fiery Glass Crusader,$41.22,9,$3.10
108,"Extraction, Quickblade Of Trembling Hands",$31.77,9,$4.76
82,Nirvana,$44.10,9,$3.33
19,"Pursuit, Cudgel of Necromancy",$8.16,8,$1.02


## Most Profitable Items
>- A sort of the above table

In [14]:
#Sort the above frame by total purchase value, format and show
most_popular_items_total_val_sort = most_popular_items_merge.sort_values('Total Purchase Value', ascending=False)
most_popular_items_total_val_sort.rename(columns={'Item Name_x': 'Item Name'}, inplace=True)
most_popular_items_total_val_sort['Total Purchase Value'] = \
most_popular_items_total_val_sort['Total Purchase Value'].map("${:.2f}".format)
most_popular_items_total_val_sort['Price'] = \
most_popular_items_total_val_sort['Price'].map("${:.2f}".format)
most_popular_items_total_val_sort[:5]

,Item Name,Total Purchase Value,Purchase Count,Price
Item ID,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",$50.76,12,$4.48
82,Nirvana,$44.10,9,$3.33
145,Fiery Glass Crusader,$41.22,9,$3.10
92,Final Critic,$39.04,8,$1.97
103,Singed Scalpel,$34.80,8,$1.93
